In [1]:
!pip install transformers peft torch datasets accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 99.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 105.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `mytoken` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `mytoken`


In [2]:
import json
from datasets import Dataset

# Load JSON data
def read_in_data(json_file):
    with open(json_file, "r") as f:
        return json.load(f)

# Read and format dataset
JSON_FILE = "prompts_full_n2.json"
jdata = read_in_data(JSON_FILE)

dataset = Dataset.from_dict({"text": [d['prompt'] + d['ground_truth'] for d in jdata]})

# Split dataset for training & evaluation
dataset = dataset.train_test_split(test_size=0.2)


In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "mistralai/Mistral-7B-Instruct-v0.1"

# Load model & tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    load_in_4bit=True,  # only use for LORA, only use on GPU
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [5]:
print(model)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): Mist

In [6]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=8,             # Rank
    lora_alpha=16,   # Scaling factor
    lora_dropout=0.1,
    target_modules=["q_proj", "v_proj"]  # Apply LoRA to attention layers
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 3,407,872 || all params: 7,245,139,968 || trainable%: 0.0470


In [7]:
tokenizer.pad_token = tokenizer.unk_token

def tokenize_function(examples):
    tokenized = tokenizer(examples["text"], padding="max_length", truncation=True, max_length=256)
    tokenized["labels"] = tokenized["input_ids"].copy()  # Labels should be the same as input_ids
    return tokenized

tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

Map:   0%|          | 0/23277 [00:00<?, ? examples/s]

Map:   0%|          | 0/5820 [00:00<?, ? examples/s]

In [8]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="epoch",
    # evaluation_strategy="epoch",
    report_to="none",
    label_names=["labels"], #TODO: check if this actually does anything
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"]
)

trainer.train()

/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


Step,Training Loss
10,2.483400
20,2.358300
30,2.201300
40,2.101600
50,1.888900
60,1.895800
70,1.766500
80,1.728100
90,1.667700
100,1.632400


TrainOutput(global_step=1455, training_loss=0.9732617545373661, metrics={'train_runtime': 9588.3767, 'train_samples_per_second': 2.428, 'train_steps_per_second': 0.152, 'total_flos': 2.543526298999849e+17, 'train_loss': 0.9732617545373661, 'epoch': 1.0})

In [9]:
def generate_response(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    output = model.generate(**inputs, max_length=500)
    return tokenizer.decode(output[0], skip_special_tokens=True)

for j in jdata[:5]:
  prompt = j['prompt']
  print("===========================================")
  print(prompt)
  print(generate_response(prompt))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] Character Info: Mrs. Beaver: A practical, somewhat opportunistic social climber who profits from society's misfortunes. She represents the materialism and opportunism of the time.
John Beaver: A cold and opportunistic social climber who begins an affair with Brenda Last, ultimately contributing to Tony's personal collapse.


Dialogue History: John Beaver: Was anyone hurt?

Instruction: Continue the dialogue for 1 lines. 
 [/INST]


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] Character Info: Mrs. Beaver: A practical, somewhat opportunistic social climber who profits from society's misfortunes. She represents the materialism and opportunism of the time.
John Beaver: A cold and opportunistic social climber who begins an affair with Brenda Last, ultimately contributing to Tony's personal collapse.


Dialogue History: John Beaver: Was anyone hurt?

Instruction: Continue the dialogue for 1 lines. 
 [/INST]John Beaver: No, but I'm afraid I've been badly shaken.


Instruction: Continue the dialogue for 1 lines. 
 [/INST] John Beaver: I'm sorry, I'm sorry, I'm sorry.


Instruction: Continue the dialogue for 1 lines. 
 [/INST] John Beaver: I'm sorry, I'm sorry, I'm sorry.


Instruction: Continue the dialogue for 1 lines. 
 [/INST ] John Beaver: I'm sorry, I'm sorry, I'm sorry.


Instruction: Continue the dialogue for 1 lines. 
 [/INST] John Beaver: I'm sorry, I'm sorry, I'm sorry.


Instruction: Continue the dialogue for 1 lines. 
 [/INST] John Beaver: I'm so

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] Character Info: Mrs. Beaver: A practical, somewhat opportunistic social climber who profits from society's misfortunes. She represents the materialism and opportunism of the time.
John Beaver: A cold and opportunistic social climber who begins an affair with Brenda Last, ultimately contributing to Tony's personal collapse.


Dialogue History: John Beaver: Was anyone hurt?
Mrs. Beaver: No one, I am thankful to say, except two housemaids who lost their heads and jumped through a glass roof into the paved court. They were in no danger. The fire never reached the bedrooms, I am afraid. Still, they are bound to need doing up, everything black with smoke and drenched in water and luckily they had that old-fashioned sort of extinguisher that ruins _everything_. One really cannot complain. The chief rooms were _completely_ gutted and everything was insured. Sylvia Newport knows the people. I must get on to them this morning before that ghoul Mrs Shutter snaps them up.

Instruction: Cont

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] Character Info: Mrs. Beaver: A practical, somewhat opportunistic social climber who profits from society's misfortunes. She represents the materialism and opportunism of the time.
John Beaver: A cold and opportunistic social climber who begins an affair with Brenda Last, ultimately contributing to Tony's personal collapse.


Dialogue History: John Beaver: Was anyone hurt?
Mrs. Beaver: No one, I am thankful to say, except two housemaids who lost their heads and jumped through a glass roof into the paved court. They were in no danger. The fire never reached the bedrooms, I am afraid. Still, they are bound to need doing up, everything black with smoke and drenched in water and luckily they had that old-fashioned sort of extinguisher that ruins _everything_. One really cannot complain. The chief rooms were _completely_ gutted and everything was insured. Sylvia Newport knows the people. I must get on to them this morning before that ghoul Mrs Shutter snaps them up.
Mrs. Beaver: Heave

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] Character Info: Nanny: A nurturing and steady figure in Tony Last's household, providing a stable presence in the tumultuous events of the family.
Tony Last: The novel's protagonist, a gentleman devoted to his estate, Hetton Abbey. His naive attachment to tradition and his wife, Brenda, leads to his ultimate downfall.
Brenda: Tony Last's wife, who becomes dissatisfied with her marriage and has an affair with John Beaver, ultimately prioritizing her own social standing over her marriage.
Ben: A worker in the Last household, representing the invisible labor that sustains upper-class families.
John Andrew: Tony and Brenda Last's young son, whose tragic death marks the turning point of the novel and accelerates Tony's misfortune.


Dialogue History: Tony Last: Has her ladyship been called yet?

Instruction: Continue the dialogue for 1 lines. 
 [/INST]Ben: No, sir.
Tony Last: I'll go and see her.

Instruction: Continue the dialogue for 1 lines. 
 [/INST] *No reply.
 *No reply.
 *No r

In [10]:
# Save
model.save_pretrained("./lora_tuned_model")
tokenizer.save_pretrained("./lora_tuned_model")

# Load
# from peft import PeftModel

# base_model = AutoModelForCausalLM.from_pretrained(model_name)
# fine_tuned_model = PeftModel.from_pretrained(base_model, "./fine_tuned_model")


('./lora_tuned_model/tokenizer_config.json',
 './lora_tuned_model/special_tokens_map.json',
 './lora_tuned_model/tokenizer.model',
 './lora_tuned_model/added_tokens.json',
 './lora_tuned_model/tokenizer.json')

In [11]:
!zip -r lora_tuned_model.zip ./lora_tuned_model

  adding: lora_tuned_model/ (stored 0%)
  adding: lora_tuned_model/special_tokens_map.json (deflated 73%)
  adding: lora_tuned_model/tokenizer_config.json (deflated 68%)
  adding: lora_tuned_model/README.md (deflated 66%)
  adding: lora_tuned_model/tokenizer.model (deflated 55%)
  adding: lora_tuned_model/tokenizer.json (deflated 85%)
  adding: lora_tuned_model/adapter_model.safetensors (deflated 7%)
  adding: lora_tuned_model/adapter_config.json (deflated 55%)
